In [8]:
import requests
import re 
import pandas as pd
from io import StringIO
import geopandas as gpd
import datetime

from bs4 import BeautifulSoup

In [2]:
closures_page= "https://www.sdbeachinfo.com/Home/GetTargetByID"
response = requests.post(closures_page)
data = response.json()
closures_df = pd.DataFrame(data)
gs=gpd.GeoSeries.from_xy(closures_df['Longitude'],closures_df['Latitude'])
closures_gdf=gpd.GeoDataFrame(closures_df, geometry=gs ,crs='EPSG:4326')
closures_gdf['Icon']= 'place'
closures_gdf['RBGColor']= closures_gdf['RBGColor'].map(lambda x: x.replace('.png',''))
closures_gdf

,SiteID,DehID,Name,Latitude,Longitude,CityID,IndicatorID,RegionID,RBGColor,RegionName,...,IndicateDescription,Description,Advisory,Closure,GeneralAdvisoryMessage,Active,StationID,TypeID,geometry,Icon
0,2,IB-010,Border Field State Park - N of fence (IB-010),32.535950,-117.124300,35,1,3,Red,None,...,None,<p><strong>Closure: Tijuana Slough Shoreline</...,"<p><span style=""font-size:14px""><strong>Closur...","<p><span style=""font-size:14px""><strong>Closur...",,True,None,1,POINT (-117.1243 32.53595),place
1,3,IB-020,Border Field State Park at Monument Road (IB-020),32.543410,-117.125228,35,1,3,Red,None,...,None,<p><strong>Closure: Tijuana Slough Shoreline</...,"<p><span style=""font-size:14px""><strong>Closur...","<p><span style=""font-size:14px""><strong>Closur...",,True,None,1,POINT (-117.12523 32.54341),place
2,4,IB-030,Tijuana Slough at Tijuana Rivermouth (IB-030),32.555139,-117.129323,35,1,3,Red,None,...,None,<p><strong>Closure: Tijuana Slough Shoreline</...,"<p><span style=""font-size:14px""><strong>Closur...","<p><span style=""font-size:14px""><strong>Closur...",,True,None,1,POINT (-117.12932 32.55514),place
3,5,IB-040,Tijuana Slough - 3/4mi N of TJ River (IB-040),32.562869,-117.132590,35,1,3,Red,None,...,None,"<p style=""margin-left:0in; margin-right:0in"">T...","<p><span style=""font-size:14px""><strong>Closur...","<p><span style=""font-size:14px""><strong>Closur...",,True,None,1,POINT (-117.13259 32.56287),place
4,6,IB-050,"Seacoast Drive, south - Imperial Beach (IB-050)",32.566577,-117.133205,35,1,3,Red,None,...,None,<p>This beach is sampled year-round (by the Co...,<p><strong>Closure: Imperial Beach Shoreline</...,<p><strong>Closure: Imperial Beach Shoreline</...,,True,None,1,POINT (-117.13321 32.56658),place
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340,471,EH-090,Crown Cove (EH-090),32.636515,-117.141163,31,2,3,Green,None,...,None,<p>This beach is sampled&nbsp;year round (by t...,,,,True,None,1,POINT (-117.14116 32.63652),place
341,472,MB-015,Enchanted Cove - Mission Bay (MB-015),32.777266,-117.216636,21,2,2,Green,None,...,None,<p>This beach is sampled during the summer sea...,<p><strong>Advisory:</strong> Mission Bay<br /...,<p><strong>Advisory:</strong> Mission Bay<br /...,,True,None,1,POINT (-117.21664 32.77727),place
342,474,SE-060,San Elijo Lagoon outlet (SE-060),33.015700,-117.281912,11,2,1,Green,None,...,None,<p>This beach is sampled weekly year-round. Ho...,<p><strong>Advisory</strong>:&nbsp;Cardiff&nbs...,<p><strong>Advisory</strong>:&nbsp;Cardiff&nbs...,,True,None,1,POINT (-117.28191 33.0157),place
343,480,MB-226,Mission Point Park (MB-226),32.760746,-117.247827,21,2,2,Green,None,...,None,<p>This beach is sampled weekly (by the County...,,,,True,None,1,POINT (-117.24783 32.76075),place


In [17]:
def parseNote(message):
    if message is None:
        return None
    pattern = re.compile(r"<strong>(.*?)</strong>", re.DOTALL)
    matches = pattern.findall(message)
    print(matches)
    if matches is not None and len(matches) > 0:
        statusNote = matches[len(matches)- 1]
    else:
        statusNote="";
    
    return statusNote
def parseSince(message):
    date_str = ""
    pattern=re.compile(
        r"<strong>Status Since:\s*</strong>\s*(?:&nbsp;)*\s*([A-Za-z]+\s+\d{1,2},\s*\d{4})(?:&nbsp;)*",
        re.IGNORECASE
    )
    match = pattern.search(message)
    if match:
        date_str = match.group(1)
        
        dt = datetime.datetime.strptime(date_str, "%B %d, %Y")
        date_str = dt.strftime("%b {day}, %Y").format(day=dt.day)
        
    else:
        print("No date found.")
    
    return date_str
def parseSdBeachinfoRow( row):
    status = ""
    statusNote = ""
    
    if row['IndicatorID']== 1 and (row['Closure'] is not None and row['Closure'] !=''):
        
        status = parseSince(row['Closure'])
        statusNote = parseNote(row['Closure'])
    elif row['IndicatorID'] == 3 and (row['Advisory'] is not None and row['Advisory'] != ''):
        
        status = parseSince(row['Advisory'])
        statusNote = parseNote(row['Advisory'])
    return [status, statusNote]

closures_gdf[['StatusSince','StatusNote']] = closures_gdf.apply(parseSdBeachinfoRow,axis=1, result_type='expand')

['Closure: Tijuana Slough Shoreline', 'Stations', 'Status Since: ', '<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/DEHQ_bb_public_summary.pdf">Bacteria levels exceed health standards. Avoid water contact in the closure&nbsp;area.</a>']
['Closure: Tijuana Slough Shoreline', 'Stations', 'Status Since: ', '<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/DEHQ_bb_public_summary.pdf">Bacteria levels exceed health standards. Avoid water contact in the closure&nbsp;area.</a>']
['Closure: Tijuana Slough Shoreline', 'Stations', 'Status Since: ', '<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/DEHQ_bb_public_summary.pdf">Bacteria levels exceed health standards. Avoid water contact in the closure&nbsp;area.</a>']
['Closure: Tijuana Slough Shoreline', 'Stations', 'Status Since: ', '<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/DEHQ_bb_public_summary.pdf">Bacteria levels excee

In [14]:
distinct_closure_html = closures_gdf['StatusNote'].unique()
distinct_closure_html

array(['<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/DEHQ_bb_public_summary.pdf">Bacteria levels exceed health standards. Avoid water contact in the closure&nbsp;area.</a>',
       'Bacteria levels exceed health standards. Avoid water contact in the closure areas.&nbsp;',
       '<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/DEHQ_bb_public_summary.pdf">Bacteria levels exceed health standards. Avoid water contact in the advisory area.</a>',
       '',
       '<a href="https://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach%26Bay/DEHQ_bb_advisory_explanation.pdf">Bacteria levels may exceed health standards. Avoid water contact in the advisory area.</a>',
       None,
       '<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/DEHQ_bb_public_summary.pdf">Bacteria levels may exceed health standards. Avoid water contact in the advisory area.</a>',
       '<u><a href="http://www.sandiegocounty.go

In [31]:
closures_gdf[closures_gdf['StatusNote']!= ''].groupby( ['StatusNote','IndicatorID'], as_index=False)['DehID'].unique()

,StatusNote,IndicatorID,DehID
0,"<a href=""http://www.sandiegocounty.gov/content...",3,[EH-120]
1,"<a href=""http://www.sandiegocounty.gov/content...",3,[MB-041]
2,"<a href=""http://www.sandiegocounty.gov/content...",1,"[IB-010, IB-020, IB-030, IB-040]"
3,"<a href=""http://www.sandiegocounty.gov/content...",3,[EH-310]
4,Bacteria levels exceed health standards. Avoid...,1,"[IB-050, EH-010, EH-030, IB-060]"


In [29]:
outfalls_text_df = closures_gdf[closures_gdf['IndicatorID']==4].groupby( ['Description','IndicatorID'], as_index=False)['DehID'].unique()
outfalls_text_df 

,Description,IndicatorID,DehID
0,"<p>\r\n\t<span style=""font-size: 11px;"">This o...",4,"[SI-030, SI-020]"
1,<p>\r\n\tAgua Hedionda Lagoon flushes with the...,4,[EH-460]
2,<p>\r\n\tBatiquitos&nbsp;Lagoon flushes with t...,4,[EH-440]
3,<p>\r\n\tBeach users are urged to avoid contac...,4,"[OC-100, MB-041, MB-203, MB-031, MB-173, MB-16..."
4,<p>\r\n\tBeach users are urged to avoid contac...,4,"[OC-100, EH-510, OC-050, OC-040, OC-020, OC-01..."
5,<p>\r\n\tBuena Vista Lagoon&nbsp;rarely flows ...,4,[EH-480]
6,<p>\r\n\tCottonwood Creek typically flows rear...,4,[EH-420]
7,<p>\r\n\tEncina Creek rarely flows in the summ...,4,[EN-030]
8,<p>\r\n\tLoma Alta&nbsp;Creek rarely flows in ...,4,[OC-022]
9,<p>\r\n\tLos Penasquitos Lagoon&nbsp;typically...,4,[FM-100]


In [69]:
def removeLink(html_string):
    link= '<a href="/">Full County Map</a>&nbsp; <strong>|</strong> &nbsp;<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_map_key_v.1.pdf">Map Legend</a>'
    linkv2='<span style="font-size:medium"><span style="color:#000000"><span style="font-family:Calibri">.</span></span><strong><span style="font-family:&quot;Calibri&quot;,sans-serif"><span style="color:#000000">&nbsp;</span><a href="/"><span style="color:#0000ff">Full County Map</span></a><span style="color:#000000">&nbsp; | &nbsp;</span><a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_maplegend.pdf"><span style="color:#0000ff">Map Legend</span></a></span></strong></span>'
    linkv3='<strong><a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_maplegend.pdf">Map Legend</a></strong>'
    linkv4='<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&Bay/bb_maplegend.pdf"><strong><span style="color:#0563c1">Map Legend</span></strong></a>'
    linkv5='<strong><a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_maplegend.pdf">Map Legend</a>&nbsp;</strong>'
    linkv6='<strong>&nbsp;<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_maplegend.pdf">Map Legend</a></strong>'
    linkv7='<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_maplegend.pdf"><strong><span style="color:#0563c1">Map Legend</span></strong></a>'
    linkv8='<strong>&nbsp;<a href="/"><span style="color:#0066cc">Full County Map</span></a>&nbsp; | &nbsp;<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_maplegend.pdf"><span style="color:#0066cc">Map Legend</span></a></strong>'
    linkv9='<strong><a href="/">Full County Map</a>&nbsp; | &nbsp;<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_maplegend.pdf">Map Legend</a></strong>'
    linkv10='<strong><a href="/"><span style="color:#0066cc">Full County Map</span></a>&nbsp; | &nbsp;<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_maplegend.pdf"><span style="color:#0066cc">Map Legend</span></a></strong>'
    link11='<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_maplegend.pdf"><strong>Map Legend</strong></a>'
    link12='<strong><a href="/"><span style="color:#0066cc">Full County Map</span></a>&nbsp; | &nbsp;<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_maplegend.pdf"><span style="color:#0066cc">Map Legend</span></a>&nbsp;</strong>'
    link13='<strong>&nbsp;<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_maplegend.pdf"><span style="color:#0066cc">Map Legend</span></a></strong>'
    link14='<strong><a href="https://admin.sdbeachinfo.com/"><span style="color:#0066cc">Full County Map</span></a>&nbsp; | <a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_maplegend.pdf"><span style="color:#0066cc">Map Legend</span></a></strong>'
    link15='<strong><a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_maplegend.pdf"><span style="color:#0066cc">Map Legend</span></a></strong>'
    link16='<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_maplegend.pdf">Map Legend</a>'
    link17='<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_maplegend.pdf"><strong>&nbsp;Map Legend</strong></a>'
    # html_string=html_string.replace(link,'').replace(linkv2,'').replace(linkv3,'').replace(linkv4,'').replace(linkv5,'').replace(linkv6,'').replace(linkv7,'').replace(linkv8,'').replace(linkv9,'').replace(linkv10,'').replace(link11,'').replace(link12,'').replace(link13,'').replace(link14,'').replace(link15,'').replace(link16,' ').replace(link17,'')
    try:
        pattern = re.compile(r'''
    (?xi)                                   # Enable case-insensitive and verbose modes.
    (?:<[^>]+>\s*)*                         # Optionally match any leading tags.
    (?:                                     # Optionally match the "Full County Map" portion:
       <a\s+href=["'][^"']*?/?["'][^>]*>      #   Match the anchor for Full County Map.
       \s*Full\s+County\s+Map\s*              #   The text (allowing whitespace variations)
       </a>
       \s*(?:&nbsp;)?\s*\|\s*(?:&nbsp;)?\s*    #   Match the separator (vertical bar with optional &nbsp;)
    )?
    <a\s+href=["'][^"']*?bb_map(?:legend|key)[^"']*?\.pdf["'][^>]*>  # Match the anchor for Map Legend.
    .*?                                     # Non-greedy match of any content inside the anchor.
    Map\s*Legend                            # Ensure the inner content contains the phrase "Map Legend" (allowing extra whitespace).
    .*?                                     # Non-greedy match until...
    </a>                                    # the closing anchor tag.
    (?:\s*</[^>]+>)*                        # Optionally match any trailing closing tags.
    ''', re.DOTALL | re.VERBOSE)

        # Remove the matched portion from the HTML string.
       # print(html_string)
        result = pattern.sub("", html_string,)
    except Exception as e:
        print(e)
        return html_string
    print ('result', result)
    return result

closures_gdf['Description'].apply(removeLink)

result <p><strong>Closure: Tijuana Slough Shoreline</strong><br />
<strong>Stations</strong>: The ocean shoreline from U.S. / Mexico border including&nbsp;Border Field State Park and the Tijuana Slough National Wildlife Refuge&nbsp;due to cross boundary flows.<br />
<strong>Status Since: </strong>December 8, 2021<br />
<u><strong><a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/DEHQ_bb_public_summary.pdf">Bacteria levels exceed health standards. Avoid water contact in the closure&nbsp;area.</a></strong></u></p>

<p><span style="font-size:11pt"><span style="font-family:Calibri">This beach&nbsp;is sampled weekly (by the county of San Diego/city of San Diego)&nbsp;&nbsp;Water quality is generally good during dry months. However, contaminated water from the Tijuana River, or other cross-border flows, can be transported north by ocean currents resulting in closures. Sample results are available from the&nbsp;<a href="https://www.waterboards.ca.gov/water_issues/p

0      <p><strong>Closure: Tijuana Slough Shoreline</...
1      <p><strong>Closure: Tijuana Slough Shoreline</...
2      <p><strong>Closure: Tijuana Slough Shoreline</...
3      <p style="margin-left:0in; margin-right:0in">T...
4      <p>This beach is sampled year-round (by the Co...
                             ...                        
340    <p>This beach is sampled&nbsp;year round (by t...
341    <p>This beach is sampled during the summer sea...
342    <p>This beach is sampled weekly year-round. Ho...
343    <p>This beach is sampled weekly (by the County...
344    <p>This beach is sampled by the County of San ...
Name: Description, Length: 345, dtype: object

In [61]:
links = {
    "link": '<a href="/">Full County Map</a>&nbsp; <strong>|</strong> &nbsp;<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_map_key_v.1.pdf">Map Legend</a>',
    "linkv2": '<span style="font-size:medium"><span style="color:#000000"><span style="font-family:Calibri">.</span></span><strong><span style="font-family:&quot;Calibri&quot;,sans-serif"><span style="color:#000000">&nbsp;</span><a href="/"><span style="color:#0066cc">Full County Map</span></a><span style="color:#000000">&nbsp; | &nbsp;</span><a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_maplegend.pdf"><span style="color:#0066cc">Map Legend</span></a></span></strong></span>',
    "linkv3": '<strong><a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_maplegend.pdf">Map Legend</a></strong>',
    "linkv4": '<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&Bay/bb_maplegend.pdf"><strong><span style="color:#0563c1">Map Legend</span></strong></a>',
    "linkv5": '<strong><a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_maplegend.pdf">Map Legend</a>&nbsp;</strong>',
    "linkv6": '<strong>&nbsp;<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_maplegend.pdf">Map Legend</a></strong>',
    "linkv7": '<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_maplegend.pdf"><strong><span style="color:#0563c1">Map Legend</span></strong></a>',
    "linkv8": '<strong>&nbsp;<a href="/"><span style="color:#0066cc">Full County Map</span></a>&nbsp; | &nbsp;<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_maplegend.pdf"><span style="color:#0066cc">Map Legend</span></a></strong>',
    "linkv9": '<strong><a href="/">Full County Map</a>&nbsp; | &nbsp;<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_maplegend.pdf">Map Legend</a></strong>',
    "linkv10": '<strong><a href="/"><span style="color:#0066cc">Full County Map</span></a>&nbsp; | &nbsp;<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_maplegend.pdf"><span style="color:#0066cc">Map Legend</span></a></strong>',
    "link11": '<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_maplegend.pdf"><strong>Map Legend</strong></a>',
    "link12": '<strong><a href="/"><span style="color:#0066cc">Full County Map</span></a>&nbsp; | &nbsp;<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_maplegend.pdf"><span style="color:#0066cc">Map Legend</span></a>&nbsp;</strong>',
    "link13": '<strong>&nbsp;<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_maplegend.pdf"><span style="color:#0066cc">Map Legend</span></a></strong>',
    "link14": '<strong><a href="https://admin.sdbeachinfo.com/"><span style="color:#0066cc">Full County Map</span></a>&nbsp; | <a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_maplegend.pdf"><span style="color:#0066cc">Map Legend</span></a></strong>',
    "link15": '<strong><a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_maplegend.pdf"><span style="color:#0066cc">Map Legend</span></a></strong>',
    "linkv16": '<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_maplegend.pdf">Map Legend</a>',
    "linkv17": '<a href="http://www.sandiegocounty.gov/content/dam/sdc/deh/lwqd/Beach&amp;Bay/bb_maplegend.pdf"><strong>&nbsp;Map Legend</strong></a>'
}

# Regular expression explanation:
# - The pattern below uses the following logic:
#   1. Optionally match a "Full County Map" link followed by a separator (e.g., “&nbsp; | &nbsp;”). This part is made optional.
#   2. Then match an <a> tag whose href attribute contains a reference to a PDF file that includes "bb_map" followed by either "legend" or "key".
#   3. Inside that <a> tag, ensure that somewhere the text "Map Legend" appears (allowing for extra tags and whitespace).
#   4. Allow optional extra wrapping tags before or after.
#
# Flags used:
# - re.IGNORECASE to make the match case-insensitive.
# - re.DOTALL so the dot (.) matches newline characters.
# - re.VERBOSE for readability.
pattern = re.compile(r'''
    (?xi)                                   # Enable case-insensitive and verbose modes.
    (?:<[^>]+>\s*)*                         # Optionally match any leading tags.
    (?:                                     # Optionally match the "Full County Map" portion:
       <a\s+href=["'][^"']*?/?["'][^>]*>      #   Match the anchor for Full County Map.
       \s*Full\s+County\s+Map\s*              #   The text (allowing whitespace variations)
       </a>
       \s*(?:&nbsp;)?\s*\|\s*(?:&nbsp;)?\s*    #   Match the separator (vertical bar with optional &nbsp;)
    )?
    <a\s+href=["'][^"']*?bb_map(?:legend|key)[^"']*?\.pdf["'][^>]*>  # Match the anchor for Map Legend.
    .*?                                     # Non-greedy match of any content inside the anchor.
    Map\s*Legend                            # Ensure the inner content contains the phrase "Map Legend" (allowing extra whitespace).
    .*?                                     # Non-greedy match until...
    </a>                                    # the closing anchor tag.
    (?:\s*</[^>]+>)*                        # Optionally match any trailing closing tags.
    ''', re.DOTALL | re.VERBOSE)

# Test the regex on all link variants
for key, value in links.items():
    match = pattern.search(value)
    print(f"{key}: {'MATCH' if match else 'NO MATCH'}")

link: NO MATCH
linkv2: MATCH
linkv3: MATCH
linkv4: MATCH
linkv5: MATCH
linkv6: MATCH
linkv7: MATCH
linkv8: MATCH
linkv9: MATCH
linkv10: MATCH
link11: MATCH
link12: MATCH
link13: MATCH
link14: MATCH
link15: MATCH
linkv16: MATCH
linkv17: MATCH
